In [ ]:
!pip install ultralytics
from ultralytics import YOLO


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import zipfile
import os

# Path to the ZIP file
zip_file_path = '/content/drive/MyDrive/Arceloproject/Construction Site Safety.v30-raw-images_latestversion.yolov8.zip'
# Directory to unzip to
extract_to_path = '/content/drive/MyDrive/Arceloproject/extracted'

# Unzipping the dataset
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

# Verify the extraction
print(os.listdir(extract_to_path))


['README.dataset.txt', 'README.roboflow.txt', 'data.yaml', 'test', 'train', 'valid']


In [ ]:
# List the contents of the extracted directory
data_yaml_path = os.path.join(extract_to_path, 'data.yaml')
if os.path.exists(data_yaml_path):
    print(f"data.yaml found at {data_yaml_path}")
else:
    print("data.yaml not found. Please check the path.")


data.yaml found at /content/drive/MyDrive/Arceloproject/extracted/data.yaml


In [ ]:
# List the contents of the extracted directory
data_yaml_path = os.path.join(extract_to_path, 'data.yaml')
if os.path.exists(data_yaml_path):
    print(f"data.yaml found at {data_yaml_path}")
else:
    print("data.yaml not found. Please check the path.")


data.yaml found at /content/drive/MyDrive/Arceloproject/extracted/data.yaml


In [ ]:
from ultralytics import YOLO

# Start training
!yolo task=detect mode=train model=yolov8l.pt data={data_yaml_path} epochs=10


100% 83.7M/83.7M [00:00<00:00, 263MB/s]
Ultralytics YOLOv8.2.41 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/content/drive/MyDrive/Arceloproject/extracted/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, sav

In [ ]:
import os

# Path to the runs directory
runs_dir = '/content/runs/detect/train/weights'

# List the contents of the runs directory
if os.path.exists(runs_dir):
    print(os.listdir(runs_dir))
else:
    print(f"{runs_dir} does not exist. Check your training logs for the actual save location.")


['best.pt', 'last.pt']


In [ ]:
import shutil

# Path to the best.pt file
best_model_path = os.path.join(runs_dir, 'best.pt')
# Path to save the model in Google Drive
drive_save_path = '/content/drive/MyDrive/Arceloproject/best.pt'

# Copy the model to Google Drive
if os.path.exists(best_model_path):
    shutil.copy(best_model_path, drive_save_path)
    print(f"Trained model saved to {drive_save_path}")
else:
    print(f"{best_model_path} does not exist. Check your training logs for the actual save location.")


Trained model saved to /content/drive/MyDrive/Arceloproject/best.pt


In [ ]:
import cv2

def create_video_writer(video_cap, output_filename):
    # grab the width, height, and fps of the frames in the video stream.
    frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))

    # initialize the FourCC and a video writer object
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    writer = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))

    return writer


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import cv2
from ultralytics import YOLO
import math

# Define create_video_writer function here

# Initialize video capture and writer
cap = cv2.VideoCapture('/content/drive/MyDrive/Arceloproject/videos/ppe-3.mp4')
output_path = '/content/drive/MyDrive/Arceloproject/ConstructionSiteSafetyOutput.mp4'
writer = create_video_writer(cap, output_path)

# Load the trained YOLO model
model = YOLO("/content/drive/MyDrive/Arceloproject/best.pt")

# Define class names
classNames = ['Excavator', 'Gloves', 'Hardhat', 'Ladder', 'Mask', 'NO-Hardhat', 'NO-Mask', 'NO-Safety Vest',
              'Person', 'SUV', 'Safety Cone', 'Safety Vest', 'bus', 'dump truck', 'fire hydrant', 'machinery',
              'mini-van', 'sedan', 'semi', 'trailer', 'truck and trailer', 'truck', 'van', 'vehicle', 'wheel loader']

# Color for bounding boxes
myColor = (0, 0, 255)

while True:
    success, img = cap.read()
    if not success:
        break

    results = model(img, stream=True)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Bounding Box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1

            # Confidence
            conf = round((box.conf[0] * 100), 2)
            # Class Name
            cls = int(box.cls[0])
            currentClass = classNames[cls]
            print(currentClass)
            if conf > 0.5:
                if currentClass == 'NO-Hardhat' or currentClass == 'NO-Safety Vest' or currentClass == "NO-Mask":
                    myColor = (0, 0, 255)
                elif currentClass == 'Hardhat' or currentClass == 'Safety Vest' or currentClass == "Mask":
                    myColor = (0, 255, 0)
                else:
                    myColor = (255, 0, 0)

                # Draw bounding box and label
                img = cv2.putText(img, f'{classNames[cls]} {conf}', (x1, y1), cv2.FONT_HERSHEY_SIMPLEX,
                                  1, (255, 0, 0), 2, cv2.LINE_AA)
                cv2.rectangle(img, (x1, y1), (x2, y2), myColor, 3)

    cv2.imshow("Image", img)
    writer.write(img)
    if cv2.waitKey(1) == ord("q"):
        break

cap.release()
writer.release()
cv2.destroyAllWindows()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
success, img = cap.read()
print(f"Video capture success: {success}")


Video capture success: False


In [ ]:
results = model(img, stream=True)
print(f"Detected objects: {results.pandas().xywh}")


WARNING ⚠️ 'source' is missing. Using 'source=/usr/local/lib/python3.10/dist-packages/ultralytics/assets'.


AttributeError: 'generator' object has no attribute 'pandas'

In [ ]:
import cv2

# Example: Loading video from Colab's local filesystem
video_path = '/content/video.mp4'
cap = cv2.VideoCapture(video_path)

# Example: Loading video from Google Drive if mounted
drive_path = '/content/drive/MyDrive/video.mp4'
cap = cv2.VideoCapture(drive_path)
if not cap.isOpened():
    print("Error: Video capture failed")
else:
    print("Video capture success")



Error: Video capture failed


In [ ]:
while True:
    success, img = cap.read()
    if not success:
        print("Error: Failed to read frame")
        break

    # Process img (perform inference, draw detections, etc.)
    cv2.imshow("Image", img)
    writer.write(img)  # Assuming writer is correctly initialized

    if cv2.waitKey(1) == ord("q"):
        break



Error: Failed to read frame


In [ ]:
cap.release()
writer.release()
cv2.destroyAllWindows()


In [ ]:
import os

# Replace with your actual path
helper_path = '/content/drive/MyDrive/Arceloproject/helper.py'

# Check if file exists
if os.path.exists(helper_path):
    print("Helper file found!")
else:
    print("Helper file not found. Check path:", helper_path)


Helper file found!


In [ ]:
output_video_path = '/content/drive/MyDrive/Arceloproject/ConstructionSiteSafetyOutput.mp4'



In [ ]:
writer = create_video_writer(cap, output_video_path)


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Adjust Python path if necessary
import sys
sys.path.append('/content/drive/MyDrive/Arceloproject')

# Import create_video_writer from helper
try:
    from helper import create_video_writer
except ImportError as e:
    print(f"Error importing create_video_writer: {e}")
    # Handle import error here


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import other necessary modules
import cv2
import math

# Define paths
video_path = '/content/drive/MyDrive/Arceloproject/videos/ppe-3.mp4'
output_video_path = '/content/drive/MyDrive/Arceloproject/ConstructionSiteSafetyOutput.mp4'

# Initialize video capture
cap = cv2.VideoCapture(video_path)

# Check if video capture is successful
if not cap.isOpened():
    print(f"Error: Unable to open video at {video_path}")
    # Print the list of files in the directory to check if the file exists
    !ls '/content/drive/MyDrive/Arceloproject/videos/'
    exit()

# Initialize video writer
writer = create_video_writer(cap, output_video_path)

while True:
    # Read a frame
    success, img = cap.read()

    # Check if frame read was successful
    if not success:
        print("Error: Frame read failed")
        break

    # Perform inference or processing on img here
    # Example: Display processed frame
    cv2.imshow("Image", img)
    writer.write(img)

    # Check for 'q' key press to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()

# Check if writer is valid before release
if writer is not None:
    writer.release()

cv2.destroyAllWindows()

Error: Unable to open video at /content/drive/MyDrive/Arceloproject/videos/ppe-3.mp4
ls: cannot access '/content/drive/MyDrive/Arceloproject/videos/': No such file or directory
Error: Frame read failed


In [ ]:
import os

video_directory = '/content/drive/MyDrive/Arceloproject/videos/'

if os.path.exists(video_directory):
    videos = os.listdir(video_directory)
    print(f"Contents of '{video_directory}':")
    print(videos)
else:
    print(f"Directory '{video_directory}' does not exist.")


Directory '/content/drive/MyDrive/Arceloproject/videos/' does not exist.
